# CI/CD for TFX pipelines

## Learning Objectives

1.  Develop a CI/CD workflow with Cloud Build to build and deploy TFX pipeline code.
2.  Integrate with Github to automatically trigger pipeline deployment with source code repository changes.

In this lab, you will walk through authoring a Cloud Build CI/CD workflow that automatically builds and deploys the same TFX pipeline from `lab-02.ipynb`. You will also integrate your workflow with GitHub by setting up a trigger that starts the workflow when a new tag is applied to the GitHub repo hosting the pipeline's code.



## Setup

In [1]:
import yaml

# Set `PATH` to include the directory containing TFX CLI.
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [2]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.25.0


**Note**: this lab was built and tested with the following package versions:

`TFX version: 0.25.0`

(Optional) If the TFX version above does not match the lab tested defaults, run the command below:

In [ ]:
%pip install --upgrade --user tfx==0.25.0

**Note**: you may need to restart the kernel to pick up the correct package versions.

## Understanding the Cloud Build workflow
Review the `cloudbuild.yaml` file to understand how the CI/CD workflow is implemented and how environment specific settings are abstracted using **Cloud Build** variables.

The **Cloud Build** CI/CD workflow automates the steps you walked through manually during `lab-02`:
1. Builds the custom TFX image to be used as a runtime execution environment for TFX components and as the AI Platform Training training container.
1. Compiles the pipeline and uploads the pipeline to the KFP environment
1. Pushes the custom TFX image to your project's **Container Registry**

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **TFX CLI**. 


## Configuring environment settings

Navigate to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

### Create or select an existing Kubernetes cluster (GKE) and deploy AI Platform

Make sure to select `"Allow access to the following Cloud APIs https://www.googleapis.com/auth/cloud-platform"` to allow for programmatic access to your pipeline by the Kubeflow SDK for the rest of the lab. Also, provide an `App instance name` such as "tfx" or "mlops". Note you may have already deployed an AI Pipelines instance during the Setup for the lab series. If so, you can proceed using that instance below in the next step.

Validate the deployment of your AI Platform Pipelines instance in the console before proceeding.

### Configure environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `GCP_REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `kubeflowpipelines-` prefix.

In [3]:
# Use the following command to identify the GCS bucket for metadata and pipeline storage.
!gsutil ls

gs://qwiklabs-gcp-00-f9d719acb080-kubeflowpipelines-default/


* `CUSTOM_SERVICE_ACCOUNT` - In the gcp console Click on the Navigation Menu and navigate to `IAM & Admin`, then to `Service Accounts` and use the service account starting with prefix - 'tfx-tuner-caip-service-account'. This enables CloudTuner and the Google Cloud AI Platform extensions Tuner component to work together and allows for distributed and parallel tuning backed by AI Platform Vizier's hyperparameter search algorithm. Please see the lab setup `README` for setup instructions.

- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. The endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console. Open the *SETTINGS* for your instance and use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window. The format is `'....[region].pipelines.googleusercontent.com'`.

In [4]:
#TODO: Set your environment resource settings here for GCP_REGION, ARTIFACT_STORE_URI, ENDPOINT, and CUSTOM_SERVICE_ACCOUNT. 

GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://qwiklabs-gcp-00-f9d719acb080-kubeflowpipelines-default'
CUSTOM_SERVICE_ACCOUNT = 'tfx-tuner-caip-service-account@qwiklabs-gcp-00-f9d719acb080.iam.gserviceaccount.com'
ENDPOINT = '15b86a1828d872d8-dot-us-central1.pipelines.googleusercontent.com'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

## Creating the TFX CLI builder

### Review the Dockerfile for the TFX CLI builder

In [5]:
!cat tfx-cli/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
COPY requirements.txt .
RUN python3 -m pip install -U -r requirements.txt

ENTRYPOINT ["tfx"]


In [6]:
!cat tfx-cli/requirements.txt

pandas>1.0.0
tfx==0.25.0
kfp==1.0.4
requests-toolbelt==0.10.1


### Build the image and push it to your project's Container Registry

**Hint**: Review the [Cloud Build](https://cloud.google.com/cloud-build/docs/running-builds/start-build-manually#gcloud) gcloud command line reference for builds submit. Your image should follow the format `gcr.io/[PROJECT_ID]/[IMAGE_NAME]:latest`. Note the source code for the tfx-cli is in the directory `./tfx-cli`.

In [7]:
IMAGE_NAME='tfx-cli'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [8]:
# TODO: Your gcloud command here to build tfx-cli and submit to Container Registry.

!gcloud builds submit --timeout=15m --tag {IMAGE_URI} tfx-cli

Creating temporary tarball archive of 2 file(s) totalling 210 bytes before compression.
Uploading tarball of [tfx-cli] to [gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042508.935873-672abb99c0e041cdbaa6925014b81e7c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-f9d719acb080/locations/global/builds/26aab0b3-500f-4f43-8c3b-75eae20f27e7].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/26aab0b3-500f-4f43-8c3b-75eae20f27e7?project=1040068750333].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "26aab0b3-500f-4f43-8c3b-75eae20f27e7"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042508.935873-672abb99c0e041cdbaa6925014b81e7c.tgz#1716042509917499
Copying gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042508.935873-672abb99c0e041cdbaa6925014b81e7c.tgz#1716042509917499...
/ [1 files][  310.0 B/  310.0 B]                      

## Exercise: manually trigger CI/CD pipeline run with Cloud Build

You can manually trigger **Cloud Build** runs using the `gcloud builds submit` command.

In [9]:
PIPELINE_NAME='tfx_covertype_continuous_training'
MODEL_NAME='tfx_covertype_classifier'
DATA_ROOT_URI='gs://cloud-training/OCBL203/workshop-datasets'
TAG_NAME='test'
TFX_IMAGE_NAME='lab-03-tfx-image'
PIPELINE_FOLDER='pipeline'
PIPELINE_DSL='runner.py'
RUNTIME_VERSION='2.3'
PYTHON_VERSION='3.7'
USE_KFP_SA='False'
ENABLE_TUNING='True'

SUBSTITUTIONS="""
_GCP_REGION={},\
_ARTIFACT_STORE_URI={},\
_CUSTOM_SERVICE_ACCOUNT={},\
_ENDPOINT={},\
_PIPELINE_NAME={},\
_MODEL_NAME={},\
_DATA_ROOT_URI={},\
_TFX_IMAGE_NAME={},\
TAG_NAME={},\
_PIPELINE_FOLDER={},\
_PIPELINE_DSL={},\
_RUNTIME_VERSION={},\
_PYTHON_VERSION={},\
_USE_KFP_SA={},\
_ENABLE_TUNING={},
""".format(GCP_REGION, 
           ARTIFACT_STORE_URI,
           CUSTOM_SERVICE_ACCOUNT,
           ENDPOINT,
           PIPELINE_NAME,
           MODEL_NAME,
           DATA_ROOT_URI,
           TFX_IMAGE_NAME,
           TAG_NAME, 
           PIPELINE_FOLDER,
           PIPELINE_DSL,
           RUNTIME_VERSION,
           PYTHON_VERSION,
           USE_KFP_SA,
           ENABLE_TUNING
           ).strip()

Hint: you can manually trigger **Cloud Build** runs using the `gcloud builds submit` command. See the [documentation](https://cloud.google.com/sdk/gcloud/reference/builds/submit) for pass the `cloudbuild.yaml` file and SUBSTITIONS as arguments.

In [10]:
# TODO: write gcloud builds submit command to trigger manual pipeline run.

!gcloud builds submit . --timeout=15m --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 13 file(s) totalling 127.3 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042913.032389-3cb274cccb444e3e8e22a8f38974061b.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-f9d719acb080/locations/global/builds/c4a336a1-e11d-49dc-a267-b90cb8a1085c].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/c4a336a1-e11d-49dc-a267-b90cb8a1085c?project=1040068750333].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "c4a336a1-e11d-49dc-a267-b90cb8a1085c"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042913.032389-3cb274cccb444e3e8e22a8f38974061b.tgz#1716042913286483
Copying gs://qwiklabs-gcp-00-f9d719acb080_cloudbuild/source/1716042913.032389-3cb274cccb444e3e8e22a8f38974061b.tgz#1716042913286483...
/ [1 files][ 26.8 KiB/ 26.8 KiB]                           

## Exercise: Setting up GitHub integration

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Create a fork of this repo

#### [Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork this repo

#### Create a Cloud Build trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location|/ workshops/tfx-caip-tf23/lab-03-tfx-cicd/labs/cloudbuild.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_GCP_REGION|[YOUR GCP_REGION]|
|_CUSTOM_SERVICE_ACCOUNT|[YOUR CUSTOM_SERVICE_ACCOUNT]|
|_ENDPOINT|[Your inverting proxy host pipeline ENDPOINT]|
|_TFX_IMAGE_NAME|lab-03-tfx-image|
|_PIPELINE_NAME|tfx_covertype_continuous_training|
|_MODEL_NAME|tfx_covertype_classifier|
|_DATA_ROOT_URI|gs://cloud-training/OCBL203/workshop-datasets|
|_PIPELINE_FOLDER|workshops/tfx-caip-tf23/lab-03-tfx-cicd/labs/pipeline|
|_PIPELINE_DSL|runner.py|
|_PYTHON_VERSION|3.7|
|_RUNTIME_VERSION|2.3|
|_USE_KFP_SA|False|
|_ENABLE_TUNING|True|

#### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


#### Verify triggered build in Cloud Build dashboard

After you see the pipeline finish building on the Cloud Build dashboard, return to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) in the console. Click `OPEN PIPELINES DASHBOARD` and view the newly deployed pipeline. Creating a release tag on GitHub will create a pipeline with the name `tfx_covertype_continuous_training-[TAG NAME]` while doing so from GitHub will create a pipeline with the name `tfx_covertype_continuous_training_github-[TAG NAME]`.

## Next Steps

In this lab, you walked through authoring a Cloud Build CI/CD workflow that automatically builds and deploys a TFX pipeline. You also integrated your TFX workflow with GitHub by setting up a Cloud Build trigger. In the next lab, you will walk through inspection of TFX metadata and pipeline artifacts created during TFX pipeline runs.

# License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>